In [ ]:
# Chubaasiny Eswararao
# Date: 3rd December 2024
# Title : Inventory and Sales Management System for Retail Shop


import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Set the directory containing the CSV files
directory = r"C:\Users\chuba\Desktop\AI\CSV_format (ARAS A, ARAS B)\CSV_format (ARAS A, ARAS B)\House B"

# List all CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Sort files to ensure the order (Day1-Day30)
csv_files.sort()

# Initialize an empty list to store dataframes
dataframes = []

# Read each file and append to the list
for file in csv_files:
    file_path = os.path.join(directory, file)
    # Specify space as the delimiter and no header since columns are not named
    df = pd.read_csv(file_path, sep='\s+', header=None)
    dataframes.append(df)

# Concatenate all dataframes into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe to a new CSV
merged_df.to_csv("HouseB_merged.csv", index=False, header=False)
print("Merged dataset saved as HouseB_merged.csv")

# Split into training and testing sets (24 days for training, 6 days for testing)
train_days = 24 * 86400  
train_df = merged_df.iloc[:train_days]
test_df = merged_df.iloc[train_days:]
print("Training and testing separated successfully")

# Separate features (sensor data) and labels
X_train = train_df.iloc[:, :-2].values  # All columns except last two (sensor data)
y_train_res1 = train_df.iloc[:, -2].values  # Second last column is activity label for resident 1
y_train_res2 = train_df.iloc[:, -1].values  # Last column is activity label for resident 2

X_test = test_df.iloc[:, :-2].values
y_test_res1 = test_df.iloc[:, -2].values
y_test_res2 = test_df.iloc[:, -1].values

# Normalize the data using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit on training data and transform both
X_test = scaler.transform(X_test)

# Reshape the data for CNN + LSTM model
# CNN expects (samples, height, width) - 3D input
# LSTM expects (samples, timesteps, features) - 3D input

# Reshape for CNN input (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # 1 feature per timestep
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Reshape for LSTM input (samples, timesteps, features)
X_train_lstm = X_train.reshape((X_train.shape[0], 20, 1))
X_test_lstm = X_test.reshape((X_test.shape[0], 20, 1))

# One-hot encode the activity labels (for both residents)
y_train_res1 = to_categorical(y_train_res1 - 1, num_classes=27)  # Adjusting for 0-indexed classes
y_test_res1 = to_categorical(y_test_res1 - 1, num_classes=27)

y_train_res2 = to_categorical(y_train_res2 - 1, num_classes=27)
y_test_res2 = to_categorical(y_test_res2 - 1, num_classes=27)

print("Preprocessing completed successfully.")


Merged dataset saved as HouseB_merged.csv
Training and testing separated successfully
Preprocessing completed successfully.


In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dropout, Dense

# CNN + LSTM Model
inputs = Input(shape=(X_train.shape[1], 1))  # Based on preprocessing reshape

# CNN Layers
Layer = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
Layer = MaxPooling1D(pool_size=2)(Layer)

# LSTM Layer (Sequence processing)
Layer = LSTM(70)(Layer)  # LSTM layer to capture the sequence dependencies
Layer = Dropout(0.1)(Layer)  # Dropout layer for regularization

# Dense Layer
Layer = Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(Layer)

# Output Layer for Resident 1 and Resident 2
out_layerR1 = Dense(27, activation='softmax', name='resident1')(Layer)
out_layerR2 = Dense(27, activation='softmax', name='resident2')(Layer)

# Define the Model
cnn_lstm_model = Model(inputs=inputs, outputs=[out_layerR1, out_layerR2])

# Compile the Model
cnn_lstm_model.compile(
    optimizer='adam',
    loss={'resident1': 'categorical_crossentropy', 'resident2': 'categorical_crossentropy'},
    loss_weights={'resident1': 0.5, 'resident2': 1.0},
    metrics={'resident1': 'accuracy', 'resident2': 'accuracy'}
)

# Model Summary
cnn_lstm_model.summary()

# Train the CNN+LSTM model for 2 Residents
cnn_lstm_model.fit(
    X_train, {'resident1': y_train_res1, 'resident2': y_train_res2}, 
    epochs=10, batch_size=64,        #   86400 * 24  = 2,073,600 / 64     = 32400
    validation_data=(X_test, {'resident1': y_test_res1, 'resident2': y_test_res2})
)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 20, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 18, 64)    │        256 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 9, 64)     │          0 │ conv1d_2[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 70)        │     37,800 │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 70)        │          0 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100)       │      7,100 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resident1 (Dense)   │ (None, 27)        │      2,727 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resident2 (Dense)   │ (None, 27)        │      2,727 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,610 (197.70 KB)

 Trainable params: 50,610 (197.70 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
32400/32400 ━━━━━━━━━━━━━━━━━━━━ 281s 9ms/step - loss: 0.6654 - resident1_accuracy: 0.9006 - resident1_loss: 0.2174 - resident2_accuracy: 0.8948 - resident2_loss: 0.4018 - val_loss: 0.5532 - val_resident1_accuracy: 0.9418 - val_resident1_loss: 0.1416 - val_resident2_accuracy: 0.9395 - val_resident2_loss: 0.3992
Epoch 2/10
32400/32400 ━━━━━━━━━━━━━━━━━━━━ 274s 7ms/step - loss: 0.5483 - resident1_accuracy: 0.9153 - resident1_loss: 0.1839 - resident2_accuracy: 0.9057 - resident2_loss: 0.3528 - val_loss: 0.5536 - val_resident1_accuracy: 0.9429 - val_resident1_loss: 0.1459 - val_resident2_accuracy: 0.9343 - val_resident2_loss: 0.3976
Epoch 3/10
32400/32400 ━━━━━━━━━━━━━━━━━━━━ 243s 7ms/step - loss: 0.5421 - resident1_accuracy: 0.9155 - resident1_loss: 0.1823 - resident2_accuracy: 0.9059 - resident2_loss: 0.3500 - val_loss: 0.5560 - val_resident1_accuracy: 0.9426 - val_resident1_loss: 0.1442 - val_resident2_accuracy: 0.9329 - val_resident2_loss: 0.4023
Epoch 4/10
32400/32400 ━━━━━

In [20]:
import time

# Start the timer
start_time = time.time()

# Evaluate the model on the test set
evaluation = cnn_lstm_model.evaluate(
    X_test, 
    {'resident1': y_test_res1, 'resident2': y_test_res2}, 
    verbose=1
)

# End the timer
end_time = time.time()
computation_time = end_time - start_time  # Time in seconds

# Extract the accuracies
accuracy_res1 = evaluation[3] * 100  # Convert to percentage
accuracy_res2 = evaluation[4] * 100  # Convert to percentage

# Loss weights for calculating the weighted average accuracy
loss_weight_res1 = 0.5
loss_weight_res2 = 1.0

# Calculate the average accuracy (weighted)
average_accuracy = (
    (accuracy_res1 * loss_weight_res1) + (accuracy_res2 * loss_weight_res2)
) / (loss_weight_res1 + loss_weight_res2)

# Print the results in the desired format
print(f"Resident 1 - Accuracy: {accuracy_res1:.2f} %")
print(f"Resident 2 - Accuracy: {accuracy_res2:.2f} %")
print(f"Average Accuracy: {average_accuracy:.2f} %")
print(f"Computation Time (seconds): {computation_time:.2f}")


16200/16200 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: 0.7767 - resident1_accuracy: 0.9542 - resident1_loss: 0.1245 - resident2_accuracy: 0.8746 - resident2_loss: 0.6440
Resident 1 - Accuracy: 94.73 %
Resident 2 - Accuracy: 93.44 %
Average Accuracy: 93.87 %
Computation Time (seconds): 38.55


In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Predict the results on the test data
y_pred_res1 = cnn_lstm_model.predict(X_test)[0]  # Predictions for Resident 1
y_pred_res2 = cnn_lstm_model.predict(X_test)[1]  # Predictions for Resident 2

# Convert the predicted values (from one-hot encoding) to class labels
y_pred_res1 = np.argmax(y_pred_res1, axis=1)
y_pred_res2 = np.argmax(y_pred_res2, axis=1)

# Convert the true labels from one-hot encoding to class labels
y_test_res1 = np.argmax(y_test_res1, axis=1)
y_test_res2 = np.argmax(y_test_res2, axis=1)

# Calculate metrics for Resident 1
precision_res1 = precision_score(y_test_res1, y_pred_res1, average='weighted', labels=np.unique(y_pred_res1)) * 100
recall_res1 = recall_score(y_test_res1, y_pred_res1, average='weighted', labels=np.unique(y_pred_res1)) * 100
f1_res1 = f1_score(y_test_res1, y_pred_res1, average='weighted', labels=np.unique(y_pred_res1)) * 100

# Calculate metrics for Resident 2
precision_res2 = precision_score(y_test_res2, y_pred_res2, average='weighted', labels=np.unique(y_pred_res2)) * 100
recall_res2 = recall_score(y_test_res2, y_pred_res2, average='weighted', labels=np.unique(y_pred_res2)) * 100
f1_res2 = f1_score(y_test_res2, y_pred_res2, average='weighted', labels=np.unique(y_pred_res2)) * 100

# Print the results in percentage
print(f"Resident 1 - Precision: {precision_res1:.2f}%, Recall: {recall_res1:.2f}%, F1 Score: {f1_res1:.2f}%")
print(f"Resident 2 - Precision: {precision_res2:.2f}%, Recall: {recall_res2:.2f}%, F1 Score: {f1_res2:.2f}%")

# Save the model after training
cnn_lstm_model.save('cnn_lstm_model.h5')  # Save the model to a file
print("Model saved as cnn_lstm_model.h5")


16200/16200 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step
16200/16200 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step


C:\Users\chuba\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Resident 1 - Precision: 94.59%, Recall: 95.05%, F1 Score: 94.66%
Resident 2 - Precision: 94.05%, Recall: 96.06%, F1 Score: 94.79%
Model saved as cnn_lstm_model.h5


In [22]:
from collections import Counter
from tabulate import tabulate

# Get the layers from the model
layers = [layer.__class__.__name__ for layer in cnn_lstm_model.layers]

# Count the number of each type of layer
layer_counts = Counter(layers)

# Prepare the data for the table
table_data = [["Type of Layers", "Number of Layers"]]
table_data.extend([[layer_type, count] for layer_type, count in layer_counts.items()])

# Print the table
print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

+------------------+--------------------+
| Type of Layers   |   Number of Layers |
+==================+====================+
| InputLayer       |                  1 |
+------------------+--------------------+
| Conv1D           |                  1 |
+------------------+--------------------+
| MaxPooling1D     |                  1 |
+------------------+--------------------+
| LSTM             |                  1 |
+------------------+--------------------+
| Dropout          |                  1 |
+------------------+--------------------+
| Dense            |                  3 |
+------------------+--------------------+
